## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [2]:
import numpy as np
import os
from random import shuffle
import re
from collections import *
from sklearn.manifold import TSNE
from gensim.models import Word2Vec

C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [4]:
import urllib.request
import zipfile
import lxml.etree

In [5]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [6]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [7]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [8]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [9]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [10]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [11]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [12]:
len(sentences_ted)

266694

In [13]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [14]:
cnt = Counter()
for sentence in sentences_ted:
    for word in sentence:
        cnt[word] += 1
        
counts_ted_top1000 = []
for common_word in cnt.most_common(1000):
    counts_ted_top1000.append(common_word[1])

Plot distribution of top-1000 words

In [15]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [16]:
model_ted = Word2Vec(sentences_ted, min_count=10)

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [25]:
model_ted.most_similar("man")

C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('guy', 0.8281731605529785),
 ('woman', 0.8246886730194092),
 ('boy', 0.7736837863922119),
 ('lady', 0.7677396535873413),
 ('girl', 0.7425094842910767),
 ('gentleman', 0.7217578887939453),
 ('soldier', 0.7185291647911072),
 ('david', 0.6913971900939941),
 ('kid', 0.6808410286903381),
 ('john', 0.6771433353424072)]

In [26]:
model_ted.most_similar("woman")

C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('girl', 0.8377739787101746),
 ('man', 0.8246886730194092),
 ('boy', 0.7962937355041504),
 ('lady', 0.7899760603904724),
 ('child', 0.7708115577697754),
 ('kid', 0.7181211709976196),
 ('soldier', 0.712570309638977),
 ('poet', 0.7117455005645752),
 ('nurse', 0.6926236152648926),
 ('gentleman', 0.688479483127594)]

In [27]:
model_ted.most_similar("dream")

C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('legacy', 0.6748108863830566),
 ('son', 0.6605381965637207),
 ('gift', 0.6435676217079163),
 ('passion', 0.6345798373222351),
 ('profession', 0.6277775764465332),
 ('career', 0.6192659735679626),
 ('plan', 0.6129658222198486),
 ('husband', 0.610859751701355),
 ('vision', 0.608635425567627),
 ('client', 0.6032742857933044)]

In [28]:
model_ted.most_similar("marriage")

C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('attorney', 0.680237352848053),
 ('minister', 0.6468159556388855),
 ('likelihood', 0.633852481842041),
 ('celebrity', 0.6262192130088806),
 ('opinion', 0.6202094554901123),
 ('abuse', 0.6183815002441406),
 ('trafficking', 0.617081880569458),
 ('equality', 0.6168765425682068),
 ('religion', 0.6109758615493774),
 ('condition', 0.6107510328292847)]

In [29]:
model_ted.most_similar("technology")

C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('knowledge', 0.7103046178817749),
 ('technologies', 0.6868748068809509),
 ('biology', 0.6831814050674438),
 ('process', 0.6690658330917358),
 ('design', 0.6683381199836731),
 ('power', 0.6637437343597412),
 ('tools', 0.663668155670166),
 ('software', 0.6611183881759644),
 ('information', 0.6535992622375488),
 ('method', 0.6526281833648682)]

In [30]:
model_ted.most_similar("entertainment")

C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('educational', 0.7439805865287781),
 ('communications', 0.7232383489608765),
 ('strategic', 0.7226549386978149),
 ('engineering', 0.7128601670265198),
 ('humanities', 0.7117301225662231),
 ('collaborative', 0.7076702117919922),
 ('exploration', 0.7045574188232422),
 ('computing', 0.7018322944641113),
 ('networking', 0.7010742425918579),
 ('robotics', 0.700797438621521)]

In [31]:
model_ted.most_similar("design")

C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('engineering', 0.7164978384971619),
 ('biology', 0.6858582496643066),
 ('model', 0.6758285760879517),
 ('technology', 0.6683381199836731),
 ('architecture', 0.6663650274276733),
 ('science', 0.6637216806411743),
 ('tool', 0.6528885364532471),
 ('mathematical', 0.643036961555481),
 ('mathematics', 0.6270281076431274),
 ('interface', 0.614954948425293)]

In [38]:
print(model_ted.most_similar_cosmul(positive=['woman', 'king'], negative=['man']),'\n')
print(model_ted.most_similar_cosmul(positive=['computer', 'mind'], negative=['man']),'\n')
print(model_ted.most_similar_cosmul(positive=['life','meaning']),'\n')
print(model_ted.most_similar_cosmul(positive=['war', 'usa'])[:3])

[('german', 0.9339950680732727), ('french', 0.9323066473007202), ('president', 0.9161515831947327), ('minister', 0.9105121493339539), ('james', 0.9089467525482178), ('mary', 0.9052067995071411), ('charles', 0.8957626819610596), ('poet', 0.8953803777694702), ('chief', 0.8933764100074768), ('robert', 0.8925814032554626)] 

[('physical', 1.342186450958252), ('testing', 1.3124470710754395), ('limits', 1.2930054664611816), ('memory', 1.253088355064392), ('biology', 1.2430435419082642), ('algorithms', 1.2386338710784912), ('communication', 1.23776113986969), ('personal', 1.237029790878296), ('processing', 1.2314305305480957), ('neural', 1.2312723398208618)] 

[('happiness', 0.6434286236763), ('experience', 0.6357074975967407), ('identity', 0.6125574707984924), ('existence', 0.6045972108840942), ('creativity', 0.6037882566452026), ('sense', 0.5993080735206604), ('imagination', 0.5988902449607849), ('purpose', 0.5968140363693237), ('passion', 0.5941535830497742), ('consciousness', 0.5939146876

C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).
  """Entry point for launching an IPython kernel.
C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).
  
C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.

#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [39]:
words_top_ted = []
for common_word in cnt.most_common(1000):
    words_top_ted.append(common_word[0])

# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted]

C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [40]:
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [41]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Extra: Literatura

In [45]:
from nltk.corpus import machado
print(machado.fileids())
print(machado.readme()) 

['contos/macn001.txt', 'contos/macn002.txt', 'contos/macn003.txt', 'contos/macn004.txt', 'contos/macn005.txt', 'contos/macn006.txt', 'contos/macn007.txt', 'contos/macn008.txt', 'contos/macn009.txt', 'contos/macn010.txt', 'contos/macn011.txt', 'contos/macn012.txt', 'contos/macn013.txt', 'contos/macn014.txt', 'contos/macn015.txt', 'contos/macn016.txt', 'contos/macn017.txt', 'contos/macn018.txt', 'contos/macn019.txt', 'contos/macn020.txt', 'contos/macn021.txt', 'contos/macn022.txt', 'contos/macn023.txt', 'contos/macn024.txt', 'contos/macn025.txt', 'contos/macn026.txt', 'contos/macn027.txt', 'contos/macn028.txt', 'contos/macn029.txt', 'contos/macn030.txt', 'contos/macn031.txt', 'contos/macn032.txt', 'contos/macn033.txt', 'contos/macn034.txt', 'contos/macn035.txt', 'contos/macn036.txt', 'contos/macn037.txt', 'contos/macn038.txt', 'contos/macn039.txt', 'contos/macn040.txt', 'contos/macn041.txt', 'contos/macn042.txt', 'contos/macn043.txt', 'contos/macn044.txt', 'contos/macn045.txt', 'contos/m

In [79]:
# Vou trabalhar com o "livro Memórias Póstumas de Brás Cubas (1881)"
raw_text = machado.raw('romance/marm05.txt')
print(raw_text)

Romance, Memórias Póstumas de Brás Cubas, 1880

Memórias Póstumas de
Brás Cubas

Texto-fonte:
Obra Completa, Machado de
Assis,
Rio
de Janeiro: Editora Nova Aguilar, 1994.

Publicado originalmente em
folhetins, a partir de março de 1880, na Revista Brasileira.

Ao verme
que
primeiro roeu as frias
carnes
do meu cadáver

dedico
como saudosa lembrança

estas
Memórias
Póstumas

Prólogo da terceira
edição

A primeira edição destas
Memórias Póstumas de Brás Cubas foi feita aos pedaços na Revista
Brasileira, pelos anos de 1880. Postas mais tarde em livro, corrigi o texto em
vários lugares. Agora que tive de o rever para a terceira edição, emendei ainda
alguma coisa e suprimi duas ou três dúzias de linhas. Assim composta, sai
novamente à luz esta obra que alguma benevolência parece ter encontrado no
público.

Capistrano de Abreu, noticiando a
publicação do livro, perguntava: As Memórias Póstumas de Brás Cubas são
um romance? Macedo Soares, em carta que me escreveu por esse tempo, recordava
am

In [57]:
text1 = machado.words('romance/marm05.txt')
print('Qtd de palavras: ', len(text1))
print('Qtd de palavras únicas: ', len(set(text1)))
print(type(text1))
print(text1[:50])

Qtd de palavras:  77098
Qtd de palavras únicas:  10848
<class 'nltk.corpus.reader.util.StreamBackedCorpusView'>
['Romance', ',', 'Memórias', 'Póstumas', 'de', 'Brás', 'Cubas', ',', '1880', 'Memórias', 'Póstumas', 'de', 'Brás', 'Cubas', 'Texto', '-', 'fonte', ':', 'Obra', 'Completa', ',', 'Machado', 'de', 'Assis', ',', 'Rio', 'de', 'Janeiro', ':', 'Editora', 'Nova', 'Aguilar', ',', '1994', '.', 'Publicado', 'originalmente', 'em', 'folhetins', ',', 'a', 'partir', 'de', 'março', 'de', '1880', ',', 'na', 'Revista', 'Brasileira']


In [62]:
sents1 = machado.sents('romance/marm05.txt')
print('Qtd de sentenças: ', len(sents1), '\n')
print(type(sents1), '\n')
print(sents1[:5])

Qtd de sentenças:  3822 

<class 'nltk.corpus.reader.util.StreamBackedCorpusView'> 

[['Romance', ',', 'Memórias', 'Póstumas', 'de', 'Brás', 'Cubas', ',', '1880'], ['Memórias', 'Póstumas', 'de', 'Brás', 'Cubas'], ['Texto', '-', 'fonte', ':', 'Obra', 'Completa', ',', 'Machado', 'de', 'Assis', ',', 'Rio', 'de', 'Janeiro', ':', 'Editora', 'Nova', 'Aguilar', ',', '1994', '.'], ['Publicado', 'originalmente', 'em', 'folhetins', ',', 'a', 'partir', 'de', 'março', 'de', '1880', ',', 'na', 'Revista', 'Brasileira', '.'], ['Ao', 'verme', 'que', 'primeiro', 'roeu', 'as', 'frias', 'carnes', 'do', 'meu', 'cadáver']]


```
for frase in range(len(sents1)):
    brasCubas = frase.replace('!','.').replace('?','.').lower()
```

---
```
with open('/home/meriat/Documents/data/copilados', 'r') as in_f:
    liter_text = in_f.read().replace('!','.').replace('?','.').lower().split('.')

print(liter_text[:3])
``` 

---

```
for s_i in range(len(liter_text)):
    liter_text[s_i] = re.sub("[^a-z]", " ", liter_text[s_i]).split()

print(liter_text[:2])
```

In [63]:
liter_text = list(sents1)

In [64]:
shuffle(liter_text)
print(len(liter_text))

3822


In [65]:
model_liter = Word2Vec(liter_text, min_count=5)

In [66]:
model_liter.most_similar("homem")

C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('meu', 0.9998600482940674),
 ('foi', 0.9998586773872375),
 ('porque', 0.9998536705970764),
 ('menos', 0.9998505115509033),
 ('estava', 0.9998486042022705),
 ('tudo', 0.9998440742492676),
 ('pouco', 0.9998428821563721),
 ('só', 0.9998383522033691),
 ('mim', 0.9998334646224976),
 ('modo', 0.9998332262039185)]

In [67]:
model_liter.most_similar("mulher")

C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('alguns', 0.9998832941055298),
 ('grande', 0.9998801946640015),
 ('dizer', 0.999873697757721),
 ('três', 0.9998721480369568),
 ('seu', 0.9998701214790344),
 ('agora', 0.999866783618927),
 ('algum', 0.9998648166656494),
 ('pela', 0.9998630285263062),
 ('anos', 0.9998628497123718),
 ('outras', 0.9998621940612793)]

In [70]:
model_liter.most_similar("crime")

C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


KeyError: "word 'roeu' not in vocabulary"

In [34]:
model_liter.most_similar("sonho")

[('pensamento', 0.7407381534576416),
 ('delirio', 0.7086445093154907),
 ('ato', 0.6971638798713684),
 ('sono', 0.6870999932289124),
 ('casamento', 0.6839599013328552),
 ('destino', 0.6808556914329529),
 ('pesadelo', 0.6767008900642395),
 ('presente', 0.670269787311554),
 ('sentido', 0.6659073233604431),
 ('periodo', 0.6622503995895386)]

In [35]:
model_liter.most_similar("amor")

[('odio', 0.7591749429702759),
 ('temor', 0.7312495708465576),
 ('sofrimento', 0.7309755682945251),
 ('orgulho', 0.7150030136108398),
 ('coracao', 0.714794933795929),
 ('sentimento', 0.7056339979171753),
 ('destino', 0.6970595717430115),
 ('espirito', 0.6963846683502197),
 ('ciume', 0.6785852909088135),
 ('desejo', 0.6772885322570801)]

In [36]:
model_liter.most_similar("morte")

[('cegueira', 0.6672433018684387),
 ('vida', 0.6591309309005737),
 ('doenca', 0.6564456224441528),
 ('existencia', 0.6374093890190125),
 ('culpa', 0.6369156241416931),
 ('desgraca', 0.6241648197174072),
 ('velhice', 0.621222734451294),
 ('salvacao', 0.6192787885665894),
 ('noticia', 0.6157522797584534),
 ('loucura', 0.6151165962219238)]

In [37]:
print(model_liter.most_similar_cosmul(positive=['roma', 'franca'], negative=['italia']), '\n')
print(model_liter.most_similar_cosmul(positive=['barco'], negative=['navegar']), '\n')
print(model_liter.most_similar_cosmul(positive=['guerra', 'paz']), '\n')
print(model_liter.most_similar_cosmul(positive=['homem'], negative=['deus']), '\n')
print(model_liter.most_similar_cosmul(positive=['labirinto', 'xadrez']), '\n')

[('napoleao', 0.9915168285369873), ('julho', 0.9496642351150513), ('paris', 0.9333292841911316), ('luis', 0.922232449054718), ('waterloo', 0.9205167293548584), ('angouleme', 0.9121435880661011), ('marco', 0.9099788069725037), ('marques', 0.8990285992622375), ('espanha', 0.8983132839202881), ('guerra', 0.8972076177597046)] 

[('quarto', 1.5034515857696533), ('jeito', 1.494570255279541), ('carro', 1.487213134765625), ('cego', 1.4491883516311646), ('comenos', 1.4336888790130615), ('menino', 1.425858497619629), ('sitio', 1.4253324270248413), ('rapaz', 1.423872709274292), ('hotel', 1.419245958328247), ('burro', 1.418635606765747)] 

[('espanha', 0.6176315546035767), ('familia', 0.6147134900093079), ('inglaterra', 0.6140139102935791), ('execucao', 0.6139072179794312), ('gloria', 0.6132325530052185), ('morte', 0.6042563915252686), ('ordem', 0.6017161011695862), ('humanidade', 0.5998936891555786), ('revolucao', 0.5978935956954956), ('salvacao', 0.5976195931434631)] 

[('usava', 1.7242372035980

In [38]:
cnt = Counter()
for sentence in liter_text:
    for word in sentence:
        cnt[word] += 1
        
counts_liter_top1000 = []
for common_word in cnt.most_common(1000):
    counts_liter_top1000.append(common_word[0])
    
words_top_vec_liter = model_liter[counts_liter_top1000]

In [39]:
tsne = TSNE(n_components=2, random_state=0)
words_top_liter_tsne = tsne.fit_transform(words_top_vec_liter)

In [40]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_liter_tsne[:,0],
                                    x2=words_top_liter_tsne[:,1],
                                    names=counts_liter_top1000))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [71]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [72]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [73]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)

for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i]).split()
    
del input_text 

In [74]:
shuffle(sentences_wiki)
print(len(sentences_wiki))

# sample 1/5 of the data
# sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
# print(len(sentences_wiki))

4267112


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [75]:
cnt = Counter()
for sentence in sentences_wiki:
    for word in sentence:
        cnt[word] += 1
        
counts_wiki_top1000 = []
for common_word in cnt.most_common(1000):
    counts_wiki_top1000.append(common_word[1])

In [76]:
hist, edges = np.histogram(counts_wiki_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

In [77]:
model_wiki = Word2Vec(sentences_wiki, min_count=10)

In [80]:
model_wiki.most_similar("man")

C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.7628993391990662),
 ('girl', 0.6610667109489441),
 ('person', 0.6573646068572998),
 ('boy', 0.6420320272445679),
 ('creature', 0.5851213932037354),
 ('kid', 0.5832953453063965),
 ('soldier', 0.5801011919975281),
 ('lad', 0.5711541175842285),
 ('dog', 0.5706813335418701),
 ('wizard', 0.5695477724075317)]

In [81]:
model_wiki.most_similar("woman")

C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('girl', 0.7911806702613831),
 ('man', 0.7628992795944214),
 ('prostitute', 0.7479172348976135),
 ('child', 0.732498288154602),
 ('person', 0.6888068914413452),
 ('lover', 0.6885553002357483),
 ('housewife', 0.6718735694885254),
 ('teenager', 0.6652871966362),
 ('herself', 0.6542807817459106),
 ('sailor', 0.6528703570365906)]

In [82]:
model_wiki.most_similar("marriage")

C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('divorce', 0.755710780620575),
 ('betrothal', 0.7162230014801025),
 ('marriages', 0.6958613395690918),
 ('marrying', 0.6791583299636841),
 ('wife', 0.6650604605674744),
 ('husband', 0.6602489948272705),
 ('daughter', 0.6343321800231934),
 ('relationship', 0.6282809376716614),
 ('marry', 0.6244655847549438),
 ('spouse', 0.6190046072006226)]

In [83]:
model_wiki.most_similar("love")

C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('smile', 0.6642180681228638),
 ('dreams', 0.6625285744667053),
 ('girl', 0.6411255598068237),
 ('kiss', 0.639606237411499),
 ('duet', 0.638232409954071),
 ('dream', 0.6352224349975586),
 ('romance', 0.6259282231330872),
 ('longing', 0.6167473793029785),
 ('sadness', 0.6094532608985901),
 ('yearning', 0.6093243360519409)]

In [84]:
print(model_wiki.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])[:3])
print(model_wiki.most_similar_cosmul(positive=['computer', 'mind'], negative=['machine'])[:3])
print(model_wiki.most_similar_cosmul(positive=['life','meaning'])[:3])
print(model_wiki.most_similar_cosmul(positive=['paris', 'japan'], negative=['france'])[:3])

[('queen', 0.9673483371734619), ('empress', 0.9655773639678955), ('monarch', 0.9483847618103027)]
[('personality', 0.9774553179740906), ('understanding', 0.9671401977539062), ('universe', 0.942608654499054)]
[('sense', 0.5263887047767639), ('essence', 0.5196608304977417), ('realizations', 0.5045477151870728)]
[('tokyo', 0.9522039294242859), ('shinjuku', 0.8906617164611816), ('osaka', 0.8839899301528931)]


C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).
  """Entry point for launching an IPython kernel.
C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).
  
C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.

In [85]:
words_top_wiki = []
for common_word in cnt.most_common(1000):
    words_top_wiki.append(common_word[0])


#### t-SNE visualization

In [86]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

C:\Users\vitor\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [87]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)